In [13]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
import torch
import torch.nn as nn
from PIL import Image
from scipy import ndimage
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
import pytorch_lightning as pl
import flash 
from flash.image import ImageClassifier
from flash.core.data.data_module import DataModule

%matplotlib inline
np.random.seed(1)

In [14]:
arr = np.random.randn(10, 5, 3)
arr = arr.transpose(2, 0, 1)
arr.shape

(3, 10, 5)

In [15]:
# Loading the data (signs)
def get_imgs_labels(h5_file_path):
    f = h5py.File(h5_file_path, "r")
    ds_keys = [key for key in f.keys()]
    imgs = np.array(f[ds_keys[1]])    
    labels = np.array(f[ds_keys[2]])
    list_classes = np.array(f[ds_keys[0]])
    imgs = np.transpose(imgs, (0, 3, 1, 2))
    return imgs, labels, list_classes

train_x, train_y, train_classes = get_imgs_labels("./datasets/train_signs.h5")
test_x, test_y, test_classes = get_imgs_labels("./datasets/test_signs.h5")
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(1080, 3, 64, 64) (1080,)
(120, 3, 64, 64) (120,)


In [16]:
# CONSTANTS

NUM_FOLDS = 5
BATCH_SIZE = 64
NUM_WORKERS = 4

In [17]:
#img = Image.fromarray(np.uint8(test_x[0])).convert('RGB')
#img

In [18]:
from flash.core.data.data_source import DataSource, DefaultDataKeys
from torchvision.datasets.folder import make_dataset
from typing import Any, Dict, Iterable , Mapping, Sequence, Callable   

class SignsDataSource(DataSource):    
    def load_data(self, h5_file_path: str) -> Sequence[Mapping[str, Any]]:
        f = h5py.File(h5_file_path, "r")
        ds_keys = [key for key in f.keys()]
        img_arr = np.array(f[ds_keys[1]])    
        label_arr = np.array(f[ds_keys[2]])        
        return [
            {
                DefaultDataKeys.INPUT: img,
                DefaultDataKeys.TARGET: label
            } 
            for img, label in list(zip(img_arr, label_arr))]

    def load_sample(self, sample: Dict[str, Any]) -> Dict[str, Any]:
        sample[DefaultDataKeys.INPUT] = Image.fromarray(np.uint8(sample[DefaultDataKeys.INPUT])).convert('RGB')
        return sample


In [19]:
from typing import Optional
from flash.core.data.process import Preprocess
from flash.core.data.data_source import DefaultDataKeys, DefaultDataSources

class SignsPreprocess(Preprocess):
    def __init__(
        self,
        train_transform: Optional[Dict[str, Callable]] = None,
        val_transform: Optional[Dict[str, Callable]] = None,
        test_transform: Optional[Dict[str, Callable]] = None,
        predict_transform: Optional[Dict[str, Callable]] = None,
    ):
        super().__init__(
            train_transform=train_transform,
            val_transform=val_transform,
            test_transform=test_transform,
            predict_transform=predict_transform,
            data_sources={
                DefaultDataSources.FILES: SignsDataSource(),
            },
            default_data_source=DefaultDataSources.FILES,
        )

    def default_transforms(self) -> Dict[str, Callable]:
        return {
            "to_tensor_transform": transforms.ToTensor(),
            "post_tensor_transform": transforms.Normalize(
                torch.tensor([0.485, 0.456, 0.406]), 
                torch.tensor([0.229, 0.224, 0.225])
                ),
            "collate": torch.utils.data._utils.collate.default_collate
        }        

    def get_state_dict(self) -> Dict[str, Any]:
        return {**self.transforms}

    @classmethod
    def load_state_dict(cls, state_dict: Dict[str, Any], strict: bool = False):
        return cls(**state_dict)

In [20]:
# for a training and label data in form of numpy arrays, return a fold_index array whose elements
# represent the fold index. The length of this fold_index array is same as length of input dataset
# and the items for which fold_index array value == cv iteration count are to be used for validation 
# in the corresponding cross validation iteration with rest of the items ( for which fold_index 
# array value != cv iteration count ) being used for training (typical ration being 80:20)
def get_skf_index(num_folds, X, y):
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = 42)
    train_fold_index = np.zeros(len(y))
    for fold, (train_index, val_index) in enumerate(skf.split(X=X, y=y)):
        train_fold_index[val_index] = [fold + 1] * len(val_index)
    return train_fold_index

k_folds = get_skf_index(num_folds=NUM_FOLDS, X=train_x, y=train_y)

In [21]:
def split_data(fold, kfolds, X, y):
    train_X = X[kfolds != fold+1]        
    train_y = y[kfolds != fold+1]    
    val_X = X[kfolds == fold+1]
    val_y = y[kfolds == fold+1]
    return train_X, train_y, val_X, val_y

In [22]:
from flash.image.classification.transforms import default_transforms
from flash.core.data.transforms import ApplyToKeys
from flash.core.data.data_source import DefaultDataKeys
from flash.image import ImageClassificationData

train_X, train_y, val_X, val_y = split_data(0, k_folds, train_x, train_y)

signs_default_transform = {
    "to_tensor_transform": nn.Sequential(
            ApplyToKeys(DefaultDataKeys.INPUT, transforms.ToTensor()),
            ApplyToKeys(DefaultDataKeys.TARGET, torch.as_tensor),
        ),
        "post_tensor_transform": ApplyToKeys(
            DefaultDataKeys.INPUT,
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ),
    "collate": torch.utils.data._utils.collate.default_collate
}

data_module = ImageClassificationData.from_numpy(
    train_data = train_X,
    train_targets= train_y,
    val_data = val_X,
    val_targets = val_y,
    predict_data = test_x,    
    train_transform = signs_default_transform,
    val_transform = signs_default_transform,    
    predict_transform = signs_default_transform,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS
)

In [23]:
item = data_module.val_dataset[10]
print(item[DefaultDataKeys.TARGET])
type(item[DefaultDataKeys.INPUT])

3


PIL.Image.Image

In [41]:
from flash.image import ImageClassifier
model = ImageClassifier(backbone="resnet18", num_classes=6 )

trainer = flash.Trainer(max_epochs=20, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=data_module, strategy="freeze_unfreeze")


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  rank_zero_deprecation(
/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | adapter       | DefaultAdapter | 11.2 M
--

Validation sanity check: 0it [00:00, ?it/s]

/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


In [42]:
predictions = model.predict(test_x, data_source=DefaultDataSources.NUMPY)

/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/flash/core/classification.py:217: UserWarning: No LabelsState was found, this serializer will act as a Classes serializer.
  rank_zero_warn("No LabelsState was found, this serializer will act as a Classes serializer.", UserWarning)


In [43]:
def test_accuracy(test_labels, test_predictions):
    incorrect_preds = []
    for index, test_label in enumerate(test_labels):
        if test_label != test_predictions[index] :
            incorrect_preds.append((test_label, test_predictions[index]))

    test_accuracy = (len(test_labels) - len(incorrect_preds)) / len(test_labels)
    return round(test_accuracy, 4), incorrect_preds

test_acc, test_incorrect_preds = test_accuracy(test_y.tolist(), predictions)
print(f"Accuracy of Resnet18 on test set is {test_acc * 100} %")    
print(f"Incorrect predictions = {len(test_incorrect_preds)} ")
for item in test_incorrect_preds:
    print(f"Actual label = {item[0]}, Predicted label = {item[1]}")

Accuracy of Resnet18 on test set is 97.5 %
Incorrect predictions = 3 
Actual label = 2, Predicted label = 1
Actual label = 2, Predicted label = 1
Actual label = 5, Predicted label = 4


In [28]:
%tensorboard --logdir lightning_logs/version_6

UsageError: Line magic function `%tensorboard` not found.
